In [1]:
import sys
import os


import MetaTrader5 as mt5
import calendar
from datetime import datetime, date
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np



physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

INPUT_SYMBOL = 'EURUSD'

IMAGE_LENGTH_M15 = 24
IMAGE_LENGTH_H1 = 24
IMAGE_LENGTH_H4 = 12
IMAGE_LENGTH_M5 = 12
IMAGE_LENGTH_D1 = 10

DAY = 2 * np.pi / (24*60*60)
WEEK = DAY / 7
YEAR = DAY / (365.2425)

THRESOLD_DIFF = 2
TARGET = 3

DAY_HISTORY_TO_PROCESS = 100

F_SINGLE_MEDIUM = True
F_VOLUME = True
F_REPEAT = 1
F_TIME = False
F_LABEL_TIMEFRAME = 'H1'
F_LABEL_PERIOD = 120
F_LABEL_TIMEFRAME_SECONDS = 900

TF_ARRAY = np.array(['H1', 'H4', 'D1', 'M15', 'M5'])

tfs = TF_ARRAY[TF_ARRAY != F_LABEL_TIMEFRAME]

_mof = []

if F_SINGLE_MEDIUM:
    _mof.append('m')
else:
    _mof.append('s')

_mof.append(str(DAY_HISTORY_TO_PROCESS))

if F_VOLUME:
    _mof.append('v1')
else:
    _mof.append('v0')

_mof.append('r' + str(F_REPEAT))

if F_TIME:
    _mof.append('t1')
else:
    _mof.append('t0')


MODELS_FOLDER = 'C:/Projects/quantsmono/Python/etfc/models/' + '_'.join(_mof) + '-' + F_LABEL_TIMEFRAME + '_' + str(F_LABEL_PERIOD) + '/'

models = [tf.keras.models.load_model(MODELS_FOLDER + 'model_0'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_1'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_2'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_3'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_4'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_5'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_6'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_7'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_8'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_9'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_10'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_11'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_12'),
          tf.keras.models.load_model(MODELS_FOLDER + 'model_13')]



In [13]:


def self_diff(a: np.ndarray, shift=1):
    return a - np.roll(a, -shift)

def build_features_one_timeframe(raw_data, timeframe):
    # raw_data: old -> new
    d = np.flipud(raw_data)
    # d: new -> old
    ts = [datetime.fromtimestamp(x) for x in d[:, 5]]
    tt = [t.timetuple() for t in ts]
    a = np.array([
        d[:, 0],
        d[:, 1],
        d[:, 2],
        d[:, 3],
        np.maximum(d[:, 0], d[:, 3]),
        np.minimum(d[:, 0], d[:, 3]),
    ], dtype=float)
    if F_SINGLE_MEDIUM:
        a = np.vstack((
            a,
            d[:, 3] - d[:, 0],
            d[:, 1] - d[:, 2],
            d[:, 1] - d[:, 0],
            d[:, 1] - d[:, 3],
            d[:, 3] - d[:, 2],
            d[:, 0] - d[:, 2]
        ))
        a = np.vstack((
            a,
            d[:, 1] - a[4],
            d[:, 1] - a[5],
            a[4] - d[:, 2],
            a[5] - d[:, 2],
            d[:, 3] + d[:, 0],
            d[:, 1] + d[:, 2],
            d[:, 1] + d[:, 0],
            d[:, 1] + d[:, 3],
            d[:, 0] + d[:, 2],
            d[:, 3] + d[:, 2],
            d[:, 1] + a[4],
            d[:, 1] + a[5],
            a[4] + d[:, 2],
            a[5] + d[:, 2],
        ))
    if F_VOLUME:
        a = np.vstack((a, d[:,4]))
    f_h_single = a.shape[0]
    for i in range(F_REPEAT):
        a = np.vstack((a, self_diff(a[i*f_h_single : (i+1)*f_h_single])))
    if F_TIME:
        a = np.vstack((
            a,
            self_diff(d[:, 5]),
            (np.sin(d[:, 5] * DAY) + 1) / 2,
            (np.cos(d[:, 5] * DAY) + 1) / 2,
            (np.sin(d[:, 5] * WEEK) + 1) / 2,
            (np.cos(d[:, 5] * WEEK) + 1) / 2,
            (np.sin(d[:, 5] * YEAR) + 1) / 2,
            (np.cos(d[:, 5] * YEAR) + 1) / 2,
            np.array([t.tm_yday for t in tt]) / 365,
            np.array([t.tm_mday for t in tt]) / 30,
            # np.array([calendar.monthrange(t.tm_year, t.tm_mon)[1] for t in tt]) / 30 - a[111], // TODO: Fix this shit
            np.array([t.tm_wday for t in tt]) / 6,
            np.array([t.tm_hour for t in tt]) / 23,
            np.array([t.tm_min for t in tt]) / 59,
        ))

    return a


def build_image(a, w):
    return [np.interp(x, (x.min(), x.max()), (0, 1)) for x in a[:,:w]]

#     M15    H1     H4     M5     D1
# ----50-----50-----20-----20-----10


In [17]:

def con_predict(raw_data_m5, raw_data_m15, raw_data_h1, raw_data_h4, raw_data_d1):
    features_m5 = build_features_one_timeframe(raw_data_m5, 'M5')
    features_m15 = build_features_one_timeframe(raw_data_m15, 'M15')
    features_h1 = build_features_one_timeframe(raw_data_h1, 'H1')
    features_h4 = build_features_one_timeframe(raw_data_h4, 'H4')
    features_d1 = build_features_one_timeframe(raw_data_d1, 'D1')

    ims = {
        'M5': build_image(features_m5, IMAGE_LENGTH_M5),
        'M15': build_image(features_m15, IMAGE_LENGTH_M15),
        'H1': build_image(features_h1, IMAGE_LENGTH_H1),
        'H4': build_image(features_h4, IMAGE_LENGTH_H4),
        'D1': build_image(features_d1, IMAGE_LENGTH_D1)
    }

    
    test_image = np.hstack((
        ims.get(F_LABEL_TIMEFRAME),
        ims.get(tfs[0]),
        ims.get(tfs[1]),
        ims.get(tfs[2]),
        ims.get(tfs[3])
    ))

    b_test_im = tf.constant([test_image])
    preds = np.array([np.argmax(models[0].predict(b_test_im), axis=-1),
                        np.argmax(models[1].predict(b_test_im), axis=-1),
                        np.argmax(models[2].predict(b_test_im), axis=-1),
                        np.argmax(models[3].predict(b_test_im), axis=-1),
                        np.argmax(models[4].predict(b_test_im), axis=-1),
                        np.argmax(models[5].predict(b_test_im), axis=-1),
                        np.argmax(models[6].predict(b_test_im), axis=-1),
                        np.argmax(models[7].predict(b_test_im), axis=-1),
                        np.argmax(models[8].predict(b_test_im), axis=-1),
                        np.argmax(models[9].predict(b_test_im), axis=-1),
                        np.argmax(models[10].predict(b_test_im), axis=-1),
                        np.argmax(models[11].predict(b_test_im), axis=-1),
                        np.argmax(models[12].predict(b_test_im), axis=-1),
                        np.argmax(models[13].predict(b_test_im), axis=-1)])


    pred_buy = preds.flat[7:].sum()
    pred_sell = preds.flat[:7].sum()

    buy_condition = (pred_buy > pred_sell + THRESOLD_DIFF) & (pred_buy >= TARGET)

    if (buy_condition):
        return 1, str(raw_data_m5[-1][5]) + " " + str(raw_data_m15[-1][5]) + " " + str(raw_data_h1[-1][5]) 
        # return 1, str(datetime.fromtimestamp(raw_data_m5[-1][5])) + " " + str(datetime.fromtimestamp(raw_data_m15[-1][5]))

    # return 0, str(datetime.fromtimestamp(raw_data_m5[-1][5])) + " " + str(datetime.fromtimestamp(raw_data_m15[-1][5]))
    return 0, str(raw_data_m5[-1][5]) + " " + str(raw_data_m15[-1][5]) + " " + str(raw_data_h1[-1][5])  + " " + str(raw_data_h4[-1][5])  + " " + str(raw_data_d1[-1][5]) 



In [15]:
from datetime import datetime
import MetaTrader5 as mt5


IMAGE_LENGTH_M15 = 24
IMAGE_LENGTH_H1 = 24
IMAGE_LENGTH_H4 = 12
IMAGE_LENGTH_M5 = 12
IMAGE_LENGTH_D1 = 10


mt5.initialize()

raw_data_m5 = mt5.copy_rates_from(INPUT_SYMBOL, mt5.TIMEFRAME_M5, datetime(2020, 7, 27, 22,55), IMAGE_LENGTH_M5 + 5)
raw_data_m15 = mt5.copy_rates_from(INPUT_SYMBOL, mt5.TIMEFRAME_M15, datetime(2020, 7, 27, 22,45), IMAGE_LENGTH_M15 + 5)
raw_data_h1 = mt5.copy_rates_from(INPUT_SYMBOL, mt5.TIMEFRAME_H1, datetime(2020, 7, 27, 22,0), IMAGE_LENGTH_H1 + 5)
raw_data_h4 = mt5.copy_rates_from(INPUT_SYMBOL, mt5.TIMEFRAME_H4, datetime(2020, 7, 27, 19,0), IMAGE_LENGTH_H4 + 5)
raw_data_d1 = mt5.copy_rates_from(INPUT_SYMBOL, mt5.TIMEFRAME_D1, datetime(2020, 7, 26, 23,0), IMAGE_LENGTH_D1 + 5)

mt5.shutdown()


def blah(x):
    rr = np.zeros(shape=(len(x), 6), dtype=float)
    rr[:,0] = x['open']
    rr[:,1] = x['high']
    rr[:,2] = x['low']
    rr[:,3] = x['close']
    rr[:,4] = x['tick_volume']
    rr[:,5] = x['time']
    return rr
    
r_m15 = blah(raw_data_m15)
r_m5 = blah(raw_data_m5)
r_h1 = blah(raw_data_h1)
r_h4 = blah(raw_data_h4)
r_d1 = blah(raw_data_d1)

In [16]:
con_predict(r_m5,r_m15,r_h1,r_h4,r_d1)

(0, '1595861700.0 1595861100.0 1595858400.0 1595836800.0 1595548800.0')